In [32]:
!pip install catboost
!pip install upgini

In [38]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=11_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace = True)
df.reset_index(inplace=True, drop=True)
df.head()


,date,store,item,sales
0,2013-01-01,2,3,19
1,2013-01-01,3,27,11
2,2013-01-01,10,21,33
3,2013-01-01,4,9,19
4,2013-01-01,7,5,5


In [40]:
train = df[df["date"] < "2017-01-01"]  # Use 2013-2016 for training
test = df[df["date"] >= "2017-01-01"]  # Use 2017 for testing

In [42]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [44]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys={"date": SearchKey.DATE},
    cv=CVType.time_series
)

enricher.fit(
    train_features, train_target,
    eval_set=[(test_features, test_target)]
)

[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
date,All valid,-
target,All valid,-




Running search request, search_id=4f5bc77e-7e47-427e-9a4a-8eb1a1a98138
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com
/

38 relevant feature(s) found with the search keys: ['date']


f_events_date_year_cos1_9014a856,3.6540,100.0000,"-0.2009, 0.4083, -0.9526",Upgini,Calendar data,Daily
f_events_date_week_sin1_847b5db1,1.5012,100.0000,"0.9749, 0.0, -0.9749",Upgini,Calendar data,Daily
f_financial_date_snp500_fdb64447,1.1564,100.0000,"1638.17, 2341.5901, 2055.47",Upgini,Markets data,Daily
f_autofe_lag_3d_35be220b,1.0801,99.5094,"0.6902, 0.3577, 0.902","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_autofe_lag_3d_d168eb3e,1.0128,99.5094,"0.0, 0.9749, -0.7818","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_autofe_lag_7d_44ccb1e1,0.7248,98.9618,"0.7162, 0.556, 0.556","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_economic_date_cci_pca_6_aa7c1005,0.6947,100.0000,"-3.5291, 5.6131, -3.4403",Upgini,World economic indicators,Daily
f_financial_date_finance_umap_0_3c020a5e,0.6597,100.0000,"9.5707, 13.0896, 13.0305",Upgini,Markets data,Daily
f_autofe_lag_7d_24084e88,0.5020,98.9618,"0.9749, 0.7818, 0.4339","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_autofe_roll_3d_min_789c0d86,0.4036,99.6805,"-0.2225, 0.6235, -0.2225","Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",Daily
f_autofe_lag_7d_6354b8dc,0.4006,98.9618,"398.75, 326.37, 335.99","Training dataset,Upgini","AutoFE: features from Training dataset,Markets data",Daily


Upgini,Calendar data,5.8765,4
"Training dataset,Upgini","AutoFE: features from Training dataset,Calendar data",5.3086,13
Upgini,Markets data,2.0771,3
Upgini,World economic indicators,1.5426,6
"Training dataset,Upgini","AutoFE: features from Training dataset,Markets data",1.4794,12


"Training dataset,Calendar data",f_autofe_lag_3d_35be220b,f_events_date_year_cos1_9014a856,lag_3d
"Training dataset,Calendar data",f_autofe_lag_3d_d168eb3e,f_events_date_week_sin1_847b5db1,lag_3d
"Training dataset,Calendar data",f_autofe_lag_7d_44ccb1e1,f_events_date_year_cos1_9014a856,lag_7d
"Training dataset,Calendar data",f_autofe_lag_7d_24084e88,f_events_date_week_sin1_847b5db1,lag_7d
"Training dataset,Calendar data",f_autofe_roll_3d_min_789c0d86,f_events_date_week_cos1_f6a8c1fc,roll_3d_min
"Training dataset,Markets data",f_autofe_lag_7d_6354b8dc,f_financial_date_stoxx_043cbcd4,lag_7d
"Training dataset,Calendar data",f_autofe_lag_2d_19eb5a25,f_events_date_week_sin1_847b5db1,lag_2d
"Training dataset,Calendar data",f_autofe_lag_2d_1947d061,f_events_date_week_cos1_f6a8c1fc,lag_2d
"Training dataset,Calendar data",f_autofe_lag_7d_1ca22899,f_events_date_week_cos3_7525fe31,lag_7d
"Training dataset,Markets data",f_autofe_roll_3d_std_8d2902a0,f_financial_date_usd_eur_5d1b9c8f,roll_3d_std
"Training dataset,Markets data",f_autofe_roll_3d_std_cf25c6ce,f_financial_date_stoxx_043cbcd4,roll_3d_std


We detected 60 outliers in your sample.
Examples of outliers with maximum value of target:
43    205
25    196
49    176
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


Train,8765,50.2364,315.812 ± 105.278,217.995 ± 52.310,97.8162
Eval 1,2235,59.5262,508.902 ± 80.995,475.020 ± 187.031,33.8827


In [50]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set=[(test_features, test_target)],
    etimator = model,
    scoring = "mean_absolute_percentage_error"
)


Calculating accuracy uplift after enrichment...
y distributions from the training sample and eval_set differ according to the Kolmogorov-Smirnov test,
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,8765,50.2364,0.286 ± 0.043,0.264 ± 0.075,0.022075
1,Eval 1,2235,59.5262,0.273 ± 0.011,0.241 ± 0.042,0.031541


In [52]:
enriched_train_features = enricher.transform(small_train_sample, keep_input=True)
enriched_test_features = enricher.transform(small_test_sample, keep_input=True)
enriched_train_features.head()

[============================================================] 100% Finished

You are trying to launch enrichment for 2 rows, which will exceed the rest limit 0.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[============================================================] 100% Finished

You are trying to launch enrichment for 2 rows, which will exceed the rest limit 0.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

AttributeError: 'NoneType' object has no attribute 'head'

In [58]:
model.fit(test_features, test_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[13.772405426706365]

In [64]:
model.fit(enriched_train_features, train_target)
enriched_preds = model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds,"SMAPE")

CatBoostError: X must not be None